# Playground V2, a Quick-Eyed Sky fork

Page
https://blog.playgroundai.com/playground-v2

Model
https://huggingface.co/playgroundai/playground-v2-1024px-aesthetic

License
https://huggingface.co/playgroundai/playground-v2-1024px-aesthetic/blob/main/LICENSE.md
<br><br>

This is a straightforward fork that incorporates the following enhancements:<b>

- All generated images are directly saved to your Google Drive.

- Dynamic Prompting (Dynamic Prompting is a simple trick that chooses randomly words in lists written like this: "Photo of a {car|train|truck} in the {sun|rain|evening|mist}).

- Settings are saved too (in a .txt), for each set.

- Batch processing of up to 200 images is now possible.

- Image dimensions have been increased to 2048 pixels.

- The Gradio interface has been expanded for a more intuitive user experience.</b><br>

You can change things (steps) directly in the code if you wish. The Huggingface page say: "Note: It is recommend to use guidance_scale=3.0."<br><br>
Click on the link when all is installed ("Running on public URL:").<br>


<br>Have fun!<br><br>

- Patreon: https://www.patreon.com/Quick_Eyed_Sky<br>
- YouTube: <b>Quick-Eyed Sky</b><br><br>

<img src="https://raw.githubusercontent.com/Quick-Eyed-Sky/Images/main/3b74994e-67aa-4118-b8b6-313feb37fb95.png" width="100%" alt="Alt text">

#🧠V6 QES_playground_v2_Navez ⚡

*if you find this helpful consider becoming a member on patreon, subscribe to my youtube for Ai applications guides*

[![Patreon](https://img.shields.io/badge/Patreon-Support-orange?style=rounded-square&logo=patreon)](https://www.patreon.com/user?u=98917275)
[![Youtube](https://img.shields.io/badge/Youtube-Subscribe-red?style=rounded-square&logo=youtube)](https://www.youtube.com/@clementetumbajulcan4712)
[![GitHub](https://img.shields.io/badge/GitHub-Visit-blue?style=rounded-square&logo=github)](https://github.com/Navezjt)
[![Huggingface](https://img.shields.io/badge/Huggingface-visit-yellow?style=rounded-square&logo=huggingface)](https://huggingface.co/JCTN)
[![Instagram](https://img.shields.io/badge/Instagarm-Follow-pink?style=rounded-square&logo=Instagram)](https://Instagram.com/joeltumbajulca)


In [ ]:
#@title # <font color="#0497e0">**JOEL CTN**</font>UI + <font color="#4684a3"> RADIO
%%html
<b>Press play on the music player(Uses only 13MB of data)</b><br/>
<audio src="http://stream.zeno.fm/vq4s9m2sg48uv" controls>

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-456b51d6-2825-7818-2552-e7766a38e292)


In [ ]:
#@title # <font color="#0497e0">**JOEL CTN**</font>UI + <font color="#4684a3"> Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#@title Installation
!pip install -q diffusers accelerate gradio

import os
import random
import uuid
from datetime import datetime

import gradio as gr
import numpy as np
from PIL import Image
import torch
from diffusers import DiffusionPipeline

folder_path = "/content/drive/My Drive/Playground"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [ ]:
#@title Image Generation Settings
DESCRIPTION = """# Quick-Eyed Sky fork of Playground v2"""
if not torch.cuda.is_available():
    DESCRIPTION += "\n<p>Running on CPU 🥶 This demo may not work on CPU.</p>"

MAX_SEED = np.iinfo(np.int32).max
CACHE_EXAMPLES = torch.cuda.is_available() and os.getenv("CACHE_EXAMPLES", "1") == "1"
MAX_IMAGE_SIZE = int(os.getenv("MAX_IMAGE_SIZE", "2048"))
USE_TORCH_COMPILE = os.getenv("USE_TORCH_COMPILE", "0") == "1"
ENABLE_CPU_OFFLOAD = os.getenv("ENABLE_CPU_OFFLOAD", "0") == "1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Problem with "secret"? Click on the key in the left menu and create a HF_TOKEN with your Hugging Face toke
# from google.colab import userdata
# userdata.get('HF_TOKEN')

if torch.cuda.is_available():
    pipe = DiffusionPipeline.from_pretrained(
        "playgroundai/playground-v2-1024px-aesthetic",
        torch_dtype=torch.float16,
        use_safetensors=True,
        add_watermarker=False,
        variant="fp16"
    )
    if ENABLE_CPU_OFFLOAD:
        pipe.enable_model_cpu_offload()
    else:
        pipe.to(device)
        print("Loaded on Device!")

    if USE_TORCH_COMPILE:
        pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
        print("Model Compiled!")

def save_image(img, folder_path):
    unique_name = str(uuid.uuid4()) + ".png"
    full_path = os.path.join(folder_path, unique_name)
    img.save(full_path)
    return full_path

def process_dynamic_prompt(prompt):
    while '{' in prompt:
        start = prompt.find('{')
        end = prompt.find('}', start)
        if end == -1:  # Just in case there's an unmatched '{'
            break
        options = prompt[start+1:end].split('|')
        choice = random.choice(options)
        prompt = prompt[:start] + choice + prompt[end+1:]
    return prompt

# Function to check if the prompt contains dynamic elements
def contains_dynamic_elements(prompt):
    return '{' in prompt and '}' in prompt

def randomize_seed_fn(seed: int, randomize_seed: bool) -> int:
    if randomize_seed:
        seed = random.randint(0, MAX_SEED)
    return seed

# Adjust the save_individual_settings function to include the original dynamic prompt
def save_individual_settings(folder_path, image_num, individual_prompt, original_dynamic_prompt, negative_prompt, seed, width, height, guidance_scale, randomize_seed):
    filename = f"settings_image_{image_num}_{str(uuid.uuid4())}.txt"
    full_path = os.path.join(folder_path, filename)
    with open(full_path, 'w') as file:
        file.write(f"Original Dynamic Prompt: {original_dynamic_prompt}\n")  # Add this line to include the original dynamic prompt
        file.write(f"Processed Prompt: {individual_prompt}\n")
        file.write(f"Negative Prompt: {negative_prompt if negative_prompt else 'None'}\n")
        file.write(f"Seed: {seed}\n")
        file.write(f"Width: {width}\n")
        file.write(f"Height: {height}\n")
        file.write(f"Guidance Scale: {guidance_scale}\n")
        file.write(f"Randomize Seed: {'Yes' if randomize_seed else 'No'}\n")
    print(f"Settings for image {image_num} saved to {full_path}")
    return individual_prompt  # Return the individual prompt for further use


def save_settings(folder_path, prompt, negative_prompt, seed, width, height, guidance_scale, num_images, randomize_seed):
    unique_id = str(uuid.uuid4())
    filename = f"settings_{unique_id}.txt"
    full_path = os.path.join(folder_path, filename)
    with open(full_path, 'w') as file:
        file.write(f"Prompt: {prompt}\n")
        file.write(f"Negative Prompt: {negative_prompt}\n")
        file.write(f"Seed: {seed}\n")
        file.write(f"Width: {width}\n")
        file.write(f"Height: {height}\n")
        file.write(f"Guidance Scale: {guidance_scale}\n")
        file.write(f"Number of Images: {num_images}\n")
        file.write(f"Randomize Seed: {randomize_seed}\n")
    print(f"Settings saved to {full_path}")


def generate(
    prompt: str,
    negative_prompt: str = "",
    use_negative_prompt: bool = False,
    seed: int = 0,
    width: int = 1024,
    height: int = 1024,
    guidance_scale: float = 7.5,
    randomize_seed: bool = False,
    num_images: int = 1,
    use_resolution_binning: bool = True,
    progress=gr.Progress(track_tqdm=True),
):
    folder_path = "/content/drive/My Drive/Playground"

    pipe.to(device)
    seed = int(randomize_seed_fn(seed, randomize_seed))
    generator = torch.Generator().manual_seed(seed)

    if not use_negative_prompt:
        negative_prompt = None

    # Check if the prompt contains dynamic elements
    dynamic_prompt = contains_dynamic_elements(prompt)

    # Save a single settings file if the prompt is not dynamic
    if not dynamic_prompt:
        save_settings(folder_path, prompt, negative_prompt, seed, width, height, guidance_scale, num_images, randomize_seed)

    image_paths = []
    for i in range(num_images):
        if dynamic_prompt:
            # Generate a unique prompt for each image if dynamic
            individual_prompt = process_dynamic_prompt(prompt)
            # Save individual settings for each image including the original dynamic prompt
            save_individual_settings(folder_path, i+1, individual_prompt, prompt, negative_prompt, seed, width, height, guidance_scale, randomize_seed)
        else:
            # Use the original prompt if not dynamic
            individual_prompt = prompt

        images = pipe(
            prompt=individual_prompt,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            guidance_scale=guidance_scale,
            num_inference_steps=25,
            generator=generator,
            num_images_per_prompt=1,
            use_resolution_binning=use_resolution_binning,
            output_type="pil",
        ).images

        for img in images:
            image_path = save_image(img, folder_path)
            image_paths.append(image_path)

    return image_paths, seed

In [ ]:
#@title Gradio Settings (when ready, click on the link "Running on Public URL")
examples = [
    "neon holography crystal cat"
]

css = '''
.gradio-container {
    max-width: 1680px !important;
}
h1 {
    text-align: center;
}
'''
with gr.Blocks(css=css) as demo:
    gr.Markdown(DESCRIPTION)
    gr.DuplicateButton(
        value="Duplicate Space for private use",
        elem_id="duplicate-button",
        visible=os.getenv("SHOW_DUPLICATE_BUTTON") == "1",
    )
    with gr.Group():
        with gr.Row():
            prompt = gr.Text(
                label="Prompt",
                show_label=False,
                placeholder="Enter your prompt",
                container=False,
            )
            run_button = gr.Button("Run", scale=0)
        result = gr.Gallery(label="Result", show_label=False)  # Removed 'columns=NUM_IMAGES_PER_PROMPT'

    with gr.Accordion("Advanced options", open=True):
        with gr.Row():
            use_negative_prompt = gr.Checkbox(label="Use negative prompt", value=True)
            negative_prompt = gr.Text(
                label="Negative prompt",
                max_lines=1,
                placeholder="Enter a negative prompt",
                visible=True,
            )
        seed = gr.Slider(
            label="Seed",
            minimum=0,
            maximum=MAX_SEED,
            step=1,
            value=0,
        )
        randomize_seed = gr.Checkbox(label="Randomize seed", value=True)
        with gr.Row(visible=True):
            width = gr.Slider(
                label="Width",
                minimum=256,
                maximum=MAX_IMAGE_SIZE,
                step=32,
                value=1024,
            )
            height = gr.Slider(
                label="Height",
                minimum=256,
                maximum=MAX_IMAGE_SIZE,
                step=32,
                value=1024,
            )
        with gr.Row():
            guidance_scale = gr.Slider(
                label="Guidance Scale",
                minimum=0.1,
                maximum=20,
                step=0.1,
                value=3.0,
            )
        num_images = gr.Slider(minimum=1, maximum=200, step=1, label="Number of Images", value=6)

    gr.Examples(
        examples=examples,
        inputs=prompt,
        outputs=[result, seed],
        fn=generate,
        cache_examples=CACHE_EXAMPLES,
    )

    use_negative_prompt.change(
        fn=lambda x: gr.update(visible=x),
        inputs=use_negative_prompt,
        outputs=negative_prompt,
        api_name=False,
    )

    gr.on(
        triggers=[
            prompt.submit,
            negative_prompt.submit,
            run_button.click,
        ],
        fn=generate,
        inputs=[
            prompt,
            negative_prompt,
            use_negative_prompt,
            seed,
            width,
            height,
            guidance_scale,
            randomize_seed,
            num_images,  # Added 'num_images' here
        ],
        outputs=[result, seed],
        api_name="run",
    )

if __name__ == "__main__":
    demo.queue(max_size=20).launch(inline=False)  # Add 'inline=False' here


My Colabs:<br><br>

•	Animagine XL 2.0: https://colab.research.google.com/drive/1U9L8MinRDl6uF6ia-6N1URIfzXpy3SGS?usp=sharing

•	SDXL-DPO: https://colab.research.google.com/drive/15f1fc6krTHaRPCRhUpWz6M0V1mnyOZtQ?usp=sharing

•	Dreamlike Photoreal 2.0: https://colab.research.google.com/drive/1ZjbbuledqapwnXVOFc6RxmTgtPi3Dd3Z?usp=sharing

•	Stable Diffusion SDXL + Kandinsky 3 + Playground V2: https://colab.research.google.com/drive/16ptLGQvzVy8V_LdNe0d0hADSQpjuUJ62?usp=sharing

•	PlayGround V2: https://colab.research.google.com/drive/1fXyZ3KiZpmUa5LVa1Vw9EfKoYtbkZLah?usp=sharing

•	Stable Video Diffusion: https://colab.research.google.com/drive/1y-vVm_PhDjpI6E_Yo6yVxYkdAfjm-6U0?usp=sharing

•	AudioCraft MusicGen: https://colab.research.google.com/drive/1eHbMw7yTvTH_8oelN86ES55upGXa2KN3?usp=sharing

•	QES MusicGEN: https://colab.research.google.com/drive/1eHbMw7yTvTH_8oelN86ES55upGXa2KN3?usp=sharing

•	GFPGAN Face Restoration: https://colab.research.google.com/drive/1ypBZ8MGFqXz3Vte-yuvCTHL-t3xTLuD3?usp=sharing

•	Image-to-Text-to-Image: https://colab.research.google.com/drive/1zsG6ruSF_Rk8Yw8OeqA0746_GVjV8Y18?usp=sharing

•	Kandinsky 2.1 + Symmetry + Dynamic Prompting + CSV + Upscaling: https://colab.research.google.com/drive/18QU-dzz4HzUcXVgVxFP_Ck1qDutXOPrv?usp=sharing

•	Kandinsky 2.1 + Automatic Outpainting: https://colab.research.google.com/drive/1e9jGVEGnvSaHqKdjzehhzSz-d4kRRYVh?usp=sharing

•	Deforum 7 + Dynamic Prompting: https://colab.research.google.com/drive/1hNmWnXnoNE9vXTujD3udJRSYTjSjowvV?usp=sharing

•	Kandinsky 3: https://colab.research.google.com/drive/13sLgeMJ_-zP7a3LIWmBEOQY3ZIk_Lhuh?usp=sharing
